In [1]:
import nltk 
nltk.download('brown')
from nltk.corpus import brown 
from scipy.sparse import csr_matrix
import scipy.sparse as sp


[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


In [2]:
corpus = brown.sents()
print(corpus)

[['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', "Atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.'], ['The', 'jury', 'further', 'said', 'in', 'term-end', 'presentments', 'that', 'the', 'City', 'Executive', 'Committee', ',', 'which', 'had', 'over-all', 'charge', 'of', 'the', 'election', ',', '``', 'deserves', 'the', 'praise', 'and', 'thanks', 'of', 'the', 'City', 'of', 'Atlanta', "''", 'for', 'the', 'manner', 'in', 'which', 'the', 'election', 'was', 'conducted', '.'], ...]


In [3]:
processed_corpus = [[word.lower() for word in sent] for sent in corpus]
#print(processed_corpus)

In [4]:
import math
word_dic = {}
word_vec_dim = 100


In [5]:
# def getRandomGaussian(u,sd,x):
#     return (math.e**((((x-u)/sd)**2)/2))/(sd*math.sqrt(2*math.pi))  


In [6]:
word_id =0
for sent in processed_corpus:
    for word in sent:
        if word in word_dic: continue
        word_dic[word] = word_id
        word_id +=1
        


In [7]:
vocabulary_size = len(word_dic)

In [8]:
import numpy as np
u = 0
sd = 1/math.sqrt(word_vec_dim)
word_embeddings = u + sd*np.random.randn(len(word_dic),word_vec_dim)

In [9]:
print(word_embeddings)

[[-0.08030225 -0.0057572   0.14385938 ... -0.0753959  -0.06330271
   0.00990981]
 [ 0.18305358 -0.00496262 -0.07639269 ...  0.06876746  0.06896138
  -0.06739364]
 [-0.06895263  0.02385585 -0.24705506 ...  0.13721291 -0.14078552
  -0.0439171 ]
 ...
 [-0.04101387  0.01836216 -0.17088447 ...  0.06655138 -0.0724738
  -0.02759786]
 [-0.16883073 -0.08946449 -0.12061808 ...  0.02054797 -0.01287911
  -0.0367249 ]
 [-0.12073965 -0.03547467  0.0042613  ...  0.04763543 -0.08877104
   0.12302579]]


In [10]:
w2 = u + sd*np.random.randn(word_vec_dim,len(word_dic))

In [11]:
print(w2.shape)

(100, 49815)


In [12]:
#Thanks ChatGPT
from scipy.sparse import csr_matrix

def gen_data_set(corpus, window, word_dic):
    X = []
    y = []
    vocabulary_size = len(word_dic)
    row = []
    col = []
    data = []
    for sent in corpus:
        for tw_index in range(len(sent)):
            target_word = sent[tw_index]
            context = sent[max(0, tw_index - window):tw_index] + sent[tw_index + 1:min(len(sent), tw_index + window + 1)]
            for cw_index in range(len(context)):
                context_word = context[cw_index]
                row.append(tw_index)
                col.append(word_dic[context_word])
                data.append(1)
    X = csr_matrix((data, (row, col)), shape=(len(corpus), vocabulary_size))
    y = [word_dic[word] for sent in corpus for word in sent]
    return X, y



In [13]:
from numba import jit, cuda

In [14]:
# window = 5
# def gen_data_set(corpus,window,word_dic):
#   X = []
#   y = []
#   vocabulary_size = len(word_dic)
#   for sent in corpus:
#       for tw_index in range(len(sent)):
#           context_vec = [0 for i in range(vocabulary_size)]
#           target_word = sent[tw_index]
#           context = sent[max(0,tw_index-window):tw_index] + sent[tw_index+1:min(len(sent),tw_index+window+1)]
#           for cw_index in range(max(0,tw_index-window),tw_index):
#               context_word = sent[cw_index]
#               context_vec[word_dic[context_word]] += 1
#           X.append(context_vec)
#           y.append(word_dic[target_word])
#           #print(context)
#           #break
#   return X,y



In [15]:
window = 5

In [16]:
X,y = gen_data_set(processed_corpus,window,word_dic)

In [17]:
print(word_embeddings.shape)

(49815, 100)


In [18]:
print(X.shape)

(57340, 49815)


In [19]:
print(vocabulary_size)

49815


In [20]:
y = np.array(y)

In [21]:
print(y.shape)

(1161192,)


In [31]:
class Layer:
    def __init__(self,weights,activation):
      self.weights = weights
      self.activation = activation
      self.out = []

class NN:
  def __init__(self,layers):
    self.layers = []
    for l in layers:
      self.layers.append(l)
  
  def activation(self,name):
    def Relu(X):
        X=X.todense()
        print("ReLU i/p shape : ", X.shape)
        return np.maximum(X,0)

    def SoftMax(X):
      #X=X.todense()
      print("X shape is ",X.shape)
      print(X)
      denom = sum(np.exp(X))
      print("Denominator is ",denom)
      out = np.exp(X)/denom
      print(sum(out))
      return out

    if name == "ReLU":
      return Relu
    elif name == "SoftMax":
      return SoftMax
    
      
  def ac_grad(self,name):
    def grad_Relu(x):
      op = (1+np.sign(x))//2
      return op
    
    def grad_SoftMax(X):
      sm = self.activation('SoftMax')(X)
      return sm*(1-sm)

    if name == 'ReLU':
      return grad_Relu

  def loss(self,name):
    def bce(y,prob_y):
      epsilon = 1e-10
      prob_y = np.clip(prob_y, epsilon, 1 - epsilon)
      N = len(y)
      # summation = 0
      # for i in range(len(y)):
      #   summation+= (y[i]*np.log(prob_y[i]) + (1-y[i])*np.log(1-prob_y[i]))
      H = (-1/N)*np.sum(y*np.log(prob_y) + (1-y)*np.log(1-prob_y))
      return H
    if name == 'BinaryCrossEntropy':
      return bce
  
  def grad(loss):
    def bce(y,prob_y):
      epsilon = 1e-10
      prob_y = np.clip(prob_y, epsilon, 1 - epsilon)
      return y/prob_y - (1-y)/(1-prob_y)
    if loss == 'BinaryCrossEntropy':
      return bce

  def forward(self,X):
    #from scipy.sparse import csr_matrix

    
    print("Forward step")
    print("Weights shape ",self.layers[0].weights.shape)
    print("Input Shape ",X.shape)
    # Convert the weight matrix to a sparse CSR matrix
    weights_sparse = csr_matrix(self.layers[0].weights)

    # Perform the sparse matrix multiplication
    print(self.layers[0].activation)
    ac = self.activation(self.layers[0].activation)
    print(type(ac))
    X=X.reshape(1,-1)
    #print(X)
    print("Input Shape ",X.shape)
    prod= np.dot(X, weights_sparse)
    #print("Product is :",prod)
    out0 = ac(prod)
    #out0 = self.activation(self.layers[0].activation)(np.dot(X,self.layers[0].weights))
    print("Out0 done")
    self.layers[0].out = out0
    print(out0.shape)
    #weights2_sparse = csr_matrix(self.layers[1].weights)
    
    out = self.activation(self.layers[1].activation)(np.dot(out0,self.layers[1].weights))
    print("Out1 done")
    self.layers[1].out = out
    return out
  
  def backward(self,X,y):
    print("Backward step")
    L = self.loss('BinaryCrossEntropy')(y,self.layers[1].out)
    dLdy = self.grad('BinaryCrossEntropy')(y,self.layers[1].out)
    dydx2 = self.ac_grad(self.layers[1].activation)(np.dot(self.layers[0].out,self.layers[1].weights))
    dLdw1 = np.dot(dLdy,np.dot(dydx2,self.layers[0].out))
    dLdy0 = np.dot(dLdy,np.dot(dydx2,self.layers[1].weights))
    dy0dx1 = self.ac_grad(self.layers[0].activation)(np.dot(X,self.layers[0].weights))
    print(X.shape, dLdy0.shape, dy0dx1.shape)
    dLdw0 = np.dot(X,np.dot(dLdy0,dy0dx1))
    
    return dLdw0, dLdw1, L
  
  def backprop(self,X,y,alpha=0.1,max_iter=1000):
    print("Beginning Backprop")
    losses = []
    for iter in range(max_iter):
      for i in range(X.shape[0]):
        row = X[i,:]
        target = y[i]
        self.forward(row)
        dLdw0, dLdw1, L = self.backward(row,target)
        self.layers[0].weights -= alpha*dLdw0
        self.layers[1].weights -= alpha*dLdw1
        losses.append(L)
        del row
        del target
        break
      if abs(L - losses[-1])< 1e-5: break 
    return losses



In [32]:
nn = NN(
    [
        Layer(word_embeddings,'ReLU'),
        Layer(w2,'SoftMax')
    ]
)

In [33]:
losses = nn.backprop(X,y,0.1,1)

Beginning Backprop
Forward step
Weights shape  (49815, 100)
Input Shape  (1, 49815)
ReLU
<class 'function'>
Input Shape  (1, 49815)
ReLU i/p shape :  (1, 100)
Out0 done
(1, 100)
X shape is  (1, 49815)
[[-2426.51544731  1211.42760386 -1924.78426155 ...  1987.21324539
  -1180.75523823   626.2717777 ]]
Denominator is  [[0.00000000e+000             inf 0.00000000e+000 ...             inf
  0.00000000e+000 9.69119268e+271]]
[[nan nan nan ... nan nan  1.]]
Out1 done
Backward step


<ipython-input-31-0ebe57d864a6>:23: RuntimeWarning: overflow encountered in exp
  denom = sum(np.exp(X))
<ipython-input-31-0ebe57d864a6>:25: RuntimeWarning: overflow encountered in exp
  out = np.exp(X)/denom
<ipython-input-31-0ebe57d864a6>:25: RuntimeWarning: invalid value encountered in true_divide
  out = np.exp(X)/denom


TypeError: ignored